# PART A.  MongoDB Data Model
---

### 'climate_record' collection example

In [ ]:
{
    "_id": ObjectId("BBB"),
    "station": "31234"
    "date" : ISODate("2017-01-20T00:00:00Z"),
    "air_temperature_celcius" : 16,
    "relative_humidity" : 50.9,
    "windspeed_knots" : 12.9,
    "max_wind_speed" : 21,
    "precipitation" : " 0.03G"
}

### 'hotspot_record' collection example

In [ ]:
{
    "_id" : ObjectId("EEE"),
    "latitude" : -38.048,
    "longitude" : 144.213,
    "datetime" : ISODate("2017-12-16T00:21:11Z"),
    "confidence" : 82,
    "date" : ISODate("2017-12-16T00:00:00Z"),
    "surface_temperature_celcius" : 55
}

### Justification

As shown in the examples above, two collections, climate_record and hotspot_record are created to model the data structure presented in the given files. The reasons are listed below:

1. There is a *one-to-many* relationship between *climate* and *hotspot*. Though mongodb prefers nested table over referencing to handle *one-to-many* relationship, it isn't practical to employ nested table in this case as there is no guarantee that climate record will always be generated before hotspot record in the live streaming. If the order of record generation is against the design, it will not be possible to store hotspot records into the system. As such, it is safer to leave the relationship as it is and rely on parent referencing to map these two tables together.

2. *station* in *climate* can be extracted out to a *Station* collection in order to achieve a 'normalized' design and hence facilitate CRUD operations on the database. In live streaming, however, in most cases there will be no updates operations. Forthermore, seperating *Station* from *Climate* imposes more processing power and time, while the rate of data processing outweight the data integrity in data streaming. As such, extracting *Station* is considered unnecessary in this scenario.

# PART B
---

### Hotspot table setup methods


In [1]:
import pymongo
import datetime


def get_hotspot_record(header, data_row):
    """
    Construct the record to insert to mongodb

    Args:
        header: A list of field names that denote the attributes of the record.
        data_row: A list of data.

    Returns:
        Record to insert

    """
    if len(header) != len(data_row):
        raise ValueError('Data might be corrupted -> header length does not match record length')

    record = {}
    for index, attribute in enumerate(header):
        attribute = attribute.strip()
        if attribute == "latitude" or attribute == "longitude":
            record[attribute] = float(data_row[index])
        elif attribute == "confidence" or attribute == "surface_temperature_celcius":
            record[attribute] = int(data_row[index])
        elif attribute == "datetime":  # convert the type of "datetime" from string to datetime
            record[attribute] = datetime.datetime.strptime(data_row[index], '%Y-%m-%dT%H:%M:%S')
        elif attribute == "date":  # convert the type of "date" from string to datetime
            record[attribute] = datetime.datetime.strptime(data_row[index], '%d/%m/%Y')
        else:
            raise ValueError("Failed to catch all the data fields")

    return record


def re_init_hotspot_table(file_location):
    """
    Drop the current hostspot table from mongodb and initialized a new one from the given file

    Args:
        file_location: CSV file where the new table is derived from

    Returns:
        whether the re-initialization is successful
    """

    data_to_insert = []   # a list of hotspot records to insert to the database

    hotspot_file = open(file_location)

    file_as_list = hotspot_file.readlines()
    header = file_as_list[0].replace("\n", "").split(",")  # get a list of field name

    for index, raw_data in enumerate(file_as_list[1:]):  # loop all rows except for the header
        data_row = raw_data.replace("\n", "").split(",")
        data_to_insert.append(get_hotspot_record(header, data_row))

    ''' database connection '''
    connection = pymongo.MongoClient("localhost", 27017)
    database = connection["fit5148_assignment_db"]
    hotspot = database["hotspot_record"]

    hotspot.drop()  # remove current data

    hotspot.insert_many(data_to_insert)
    hotspot_file.close()
    return True

### Climate and station table setup methods

In [2]:
import pymongo
import datetime

from bson import ObjectId

import pymongo
import datetime

from bson import ObjectId


def get_climate_record(header, data_row):
    """
    Construct the record to insert to mongodb

    Args:
        header: A list of field names that denote the attributes of the record.
        data_row: A list of data.

    Returns:
        Record to insert

    """
    if len(header) != len(data_row):
        raise ValueError('Data might be corrupted -> header length does not match record length')

    climate_record = {}
    for index, attribute in enumerate(header):
        attribute = attribute.strip()
        if attribute == "station" or attribute == "precipitation":
            climate_record[attribute] = str(data_row[index])
        elif attribute == "date":  # convert the type of "date" from string to datetime
            climate_record[attribute] = datetime.datetime.strptime(data_row[index], '%d/%m/%Y')
        elif attribute == "air_temperature_celcius":
            climate_record[attribute] = int(data_row[index])
        elif attribute == "relative_humidity" or attribute == "windspeed_knots" or attribute == "max_wind_speed":
            climate_record[attribute] = float(data_row[index])

        else:
            raise ValueError("Failed to catch all the data fields")

    return climate_record


def re_init_climate_table(file_location):
    """ Drop the current hostspot table from mongodb and initialized a new one from the given file

    Args:
        file_location: CSV file where the new table is derived from

    Returns:
        whether the re-initialization is successful
    """

    stations_dict = {}   # a dictionary of stations for fast station lookup
    climate_records = []

    climate_file = open(file_location)

    file_as_list = climate_file.readlines()
    header = file_as_list[0].replace("\n", "").split(",")  # get a list of field names

    for index, raw_data in enumerate(file_as_list[1:]):  # loop all rows except for the header
        data_row = raw_data.replace("\n", "").split(",")
        climate_record = get_climate_record(header, data_row)

        climate_records.append(climate_record)  # add records for climate table

    ''' database connection '''
    connection = pymongo.MongoClient("localhost", 27017)
    db = connection["fit5148_assignment_db"]

    ''' drop current tables'''
    db.climate.drop()

    ''' insert data to database'''
    db.climate.insert_many(climate_records)

    return True

### Run the statements below to initialize the database
***Please make sure your mongodb is running and the correct paths are provided***

In [4]:
re_init_hotspot_table("/Users/frank/Desktop/Sem3/FIT5148/Assignment/data/hotspot_historic.csv")
re_init_climate_table("/Users/frank/Desktop/Sem3/FIT5148/Assignment/data/climate_historic.csv")

True

### a
Find climate data on 10th December 2017 .

In [13]:
import pymongo
import datetime,time
def question_a():
    connection = pymongo.MongoClient("localhost",27017) # connet database 
    database = connection["fit5148_assignment_db"] # connect climate database
    cli_collection = database["climate"]
    dateList = [] 
    # transfer the string to date to query 
    date = datetime.datetime.strptime("2017-12-10",'%Y-%m-%d')
    result = list(cli_collection.find({"date":date}))
    return result
    
question_a()

[{'_id': ObjectId('5ce737cc869a0fa6b08876d5'),
  'station': '948702',
  'date': datetime.datetime(2017, 12, 10, 0, 0),
  'air_temperature_celcius': 17,
  'relative_humidity': 53.5,
  'windspeed_knots': 7.3,
  'max_wind_speed': 14.0,
  'precipitation': ' 0.00I'}]

### b 
Find the latitude , longitude, surface temperature (°C) , and confidence when
the surface temperature (°C) was between 65 °C and 100 °C .

In [14]:
import pymongo

def question_b():
    connection = pymongo.MongoClient("localhost", 27017)
    database = connection["fit5148_assignment_db"]
    hotspot = database["hotspot_record"]

    records = hotspot.find(
        {
            "$and": [
                {
                    "surface_temperature_celcius": {"$gte": 65}
                },
                {
                    "surface_temperature_celcius": {"$lte": 100}
                }
            ]
        },
        {
            "latitude": 1,
            "longitude": 1,
            "surface_temperature_celcius": 1,
            "confidence": 1
        }
    )

    for record in records:
        print(record)

    return records


question_b()

{'_id': ObjectId('5ce737cc869a0fa6b0886b11'), 'latitude': -37.966, 'longitude': 145.051, 'confidence': 78, 'surface_temperature_celcius': 68}
{'_id': ObjectId('5ce737cc869a0fa6b0886b14'), 'latitude': -35.543, 'longitude': 143.316, 'confidence': 86, 'surface_temperature_celcius': 67}
{'_id': ObjectId('5ce737cc869a0fa6b0886b1b'), 'latitude': -37.875, 'longitude': 142.51, 'confidence': 93, 'surface_temperature_celcius': 73}
{'_id': ObjectId('5ce737cc869a0fa6b0886b1d'), 'latitude': -37.613, 'longitude': 149.305, 'confidence': 95, 'surface_temperature_celcius': 75}
{'_id': ObjectId('5ce737cc869a0fa6b0886b1f'), 'latitude': -37.624, 'longitude': 149.314, 'confidence': 90, 'surface_temperature_celcius': 66}
{'_id': ObjectId('5ce737cc869a0fa6b0886b22'), 'latitude': -38.057, 'longitude': 144.211, 'confidence': 93, 'surface_temperature_celcius': 73}
{'_id': ObjectId('5ce737cc869a0fa6b0886b2b'), 'latitude': -37.95, 'longitude': 142.366, 'confidence': 92, 'surface_temperature_celcius': 70}
{'_id': 

### c
Find date , surface temperature (°C), air temperature (°C), relative humidity and
max wind speed on 15th and 16th of December 2017.

In [15]:
def question_d():
    connection = pymongo.MongoClient("localhost", 27017)
    database = connection["fit5148_assignment_db"]

    climate_record = database["climate"]
    date = datetime.datetime.strptime("2017-12-15",'%Y-%m-%d')
    date2 = datetime.datetime.strptime("2017-12-16",'%Y-%m-%d')
    records = climate_record.aggregate([
        {
            "$lookup": {  # join climate_records and hotspot_records
                "from": "hotspot_record",
                "localField": "date",
                "foreignField": "date",
                "as": "hotspot"
            }
        },
        {"$unwind": "$hotspot"},  # unwind in order to do a match
        {
            "$match": {  # 15th and 16th of December 2017
                "$or": [
                    {"date": date},
                    {"date": date2}
                ]
            }
        },
        {
            "$project": {  # project air temperature , surface temperature and confidence
                "hotspot.surface_temperature_celcius": 1,
                "air_temperature_celcius": 1,
                "relative_humidity": 1,
                "max_wind_speed": 1
                
            }
        }

    ])


    for record in records:
        print(record)

    return records


question_d()

{'_id': ObjectId('5ce737cc869a0fa6b08876da'), 'air_temperature_celcius': 18, 'relative_humidity': 52.0, 'max_wind_speed': 14.0, 'hotspot': {'surface_temperature_celcius': 42}}
{'_id': ObjectId('5ce737cc869a0fa6b08876da'), 'air_temperature_celcius': 18, 'relative_humidity': 52.0, 'max_wind_speed': 14.0, 'hotspot': {'surface_temperature_celcius': 36}}
{'_id': ObjectId('5ce737cc869a0fa6b08876da'), 'air_temperature_celcius': 18, 'relative_humidity': 52.0, 'max_wind_speed': 14.0, 'hotspot': {'surface_temperature_celcius': 38}}
{'_id': ObjectId('5ce737cc869a0fa6b08876da'), 'air_temperature_celcius': 18, 'relative_humidity': 52.0, 'max_wind_speed': 14.0, 'hotspot': {'surface_temperature_celcius': 40}}
{'_id': ObjectId('5ce737cc869a0fa6b08876db'), 'air_temperature_celcius': 18, 'relative_humidity': 53.7, 'max_wind_speed': 13.0, 'hotspot': {'surface_temperature_celcius': 43}}
{'_id': ObjectId('5ce737cc869a0fa6b08876db'), 'air_temperature_celcius': 18, 'relative_humidity': 53.7, 'max_wind_speed'

### d 

Find datetime, *air temperature (°C)*, surface temperature (°C) and confidence when the confidence is between 80 and 100.

In [17]:
import pymongo


def question_d():
    connection = pymongo.MongoClient("localhost", 27017)
    database = connection["fit5148_assignment_db"]

    climate_record = database["climate"]

    records = climate_record.aggregate([
        {
            "$lookup": {  # join climate_records and hotspot_records
                "from": "hotspot_record",
                "localField": "date",
                "foreignField": "date",
                "as": "hotspot"
            }
        },
        {"$unwind": "$hotspot"},  # unwind in order to do a match
        {
            "$match": {  # 80 <= confidence <= 100
                "$and": [
                    {"hotspot.confidence": {"$gte": 80}},
                    {"hotspot.confidence": {"$lte": 100}}
                ]
            }
        },
        {
            "$project": {  # project air temperature , surface temperature and confidence
                "air_temperature_celcius": 1,
                "hotspot.surface_temperature_celcius": 1,
                "hotspots.confidence": 1
            }
        }

    ])


    for record in records:
        print(record)

    return records


question_d()

{'_id': ObjectId('5ce737cc869a0fa6b08875be'), 'air_temperature_celcius': 20, 'hotspot': {'surface_temperature_celcius': 62}}
{'_id': ObjectId('5ce737cc869a0fa6b08875be'), 'air_temperature_celcius': 20, 'hotspot': {'surface_temperature_celcius': 59}}
{'_id': ObjectId('5ce737cc869a0fa6b08875bf'), 'air_temperature_celcius': 19, 'hotspot': {'surface_temperature_celcius': 64}}
{'_id': ObjectId('5ce737cc869a0fa6b08875c1'), 'air_temperature_celcius': 23, 'hotspot': {'surface_temperature_celcius': 41}}
{'_id': ObjectId('5ce737cc869a0fa6b08875c2'), 'air_temperature_celcius': 19, 'hotspot': {'surface_temperature_celcius': 105}}
{'_id': ObjectId('5ce737cc869a0fa6b08875c2'), 'air_temperature_celcius': 19, 'hotspot': {'surface_temperature_celcius': 109}}
{'_id': ObjectId('5ce737cc869a0fa6b08875c2'), 'air_temperature_celcius': 19, 'hotspot': {'surface_temperature_celcius': 73}}
{'_id': ObjectId('5ce737cc869a0fa6b08875c2'), 'air_temperature_celcius': 19, 'hotspot': {'surface_temperature_celcius': 80}

{'_id': ObjectId('5ce737cc869a0fa6b08875da'), 'air_temperature_celcius': 15, 'hotspot': {'surface_temperature_celcius': 59}}
{'_id': ObjectId('5ce737cc869a0fa6b08875da'), 'air_temperature_celcius': 15, 'hotspot': {'surface_temperature_celcius': 58}}
{'_id': ObjectId('5ce737cc869a0fa6b08875da'), 'air_temperature_celcius': 15, 'hotspot': {'surface_temperature_celcius': 90}}
{'_id': ObjectId('5ce737cc869a0fa6b08875da'), 'air_temperature_celcius': 15, 'hotspot': {'surface_temperature_celcius': 115}}
{'_id': ObjectId('5ce737cc869a0fa6b08875da'), 'air_temperature_celcius': 15, 'hotspot': {'surface_temperature_celcius': 61}}
{'_id': ObjectId('5ce737cc869a0fa6b08875da'), 'air_temperature_celcius': 15, 'hotspot': {'surface_temperature_celcius': 98}}
{'_id': ObjectId('5ce737cc869a0fa6b08875da'), 'air_temperature_celcius': 15, 'hotspot': {'surface_temperature_celcius': 89}}
{'_id': ObjectId('5ce737cc869a0fa6b08875da'), 'air_temperature_celcius': 15, 'hotspot': {'surface_temperature_celcius': 59}}

### e
Find the top 10 records with the highest surface temperature (°C).

In [18]:
import pymongo


def question_e():
    connection = pymongo.MongoClient("localhost", 27017)
    database = connection["fit5148_assignment_db"]
    hotspot = database["hotspot_record"]

    records = hotspot.find().sort("surface_temperature_celcius", direction=pymongo.DESCENDING).limit(10)

    for record in records:
        print(record)

    return records


question_e()

{'_id': ObjectId('5ce737cc869a0fa6b0886fb4'), 'latitude': -38.1665, 'longitude': 143.062, 'datetime': datetime.datetime(2017, 4, 18, 4, 52), 'confidence': 100, 'date': datetime.datetime(2017, 4, 18, 0, 0), 'surface_temperature_celcius': 124}
{'_id': ObjectId('5ce737cc869a0fa6b088745e'), 'latitude': -36.343, 'longitude': 142.1986, 'datetime': datetime.datetime(2017, 4, 4, 4, 32, 50), 'confidence': 100, 'date': datetime.datetime(2017, 4, 4, 0, 0), 'surface_temperature_celcius': 123}
{'_id': ObjectId('5ce737cc869a0fa6b0886f26'), 'latitude': -36.9318, 'longitude': 143.0907, 'datetime': datetime.datetime(2017, 5, 1, 4, 14, 20), 'confidence': 100, 'date': datetime.datetime(2017, 5, 1, 0, 0), 'surface_temperature_celcius': 122}
{'_id': ObjectId('5ce737cc869a0fa6b088754d'), 'latitude': -37.017, 'longitude': 148.1297, 'datetime': datetime.datetime(2017, 3, 18, 3, 50, 50), 'confidence': 100, 'date': datetime.datetime(2017, 3, 18, 0, 0), 'surface_temperature_celcius': 121}
{'_id': ObjectId('5ce73

### f
Find the number of fire in each day. You are required to only display the
total number of fire and the date in the output.

In [19]:
import pymongo


def question_f():
    connection = pymongo.MongoClient("localhost", 27017)
    database = connection["fit5148_assignment_db"]
    hotspot = database["hotspot_record"]

    records = hotspot.aggregate([
        {"$group": {"_id": "$date", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ])

    for record in records:
        print(record)

    return records


question_f()

{'_id': datetime.datetime(2017, 4, 13, 0, 0), 'count': 357}
{'_id': datetime.datetime(2017, 4, 18, 0, 0), 'count': 325}
{'_id': datetime.datetime(2017, 5, 4, 0, 0), 'count': 135}
{'_id': datetime.datetime(2017, 4, 6, 0, 0), 'count': 118}
{'_id': datetime.datetime(2017, 5, 10, 0, 0), 'count': 114}
{'_id': datetime.datetime(2017, 5, 15, 0, 0), 'count': 102}
{'_id': datetime.datetime(2017, 4, 4, 0, 0), 'count': 89}
{'_id': datetime.datetime(2017, 4, 3, 0, 0), 'count': 72}
{'_id': datetime.datetime(2017, 4, 12, 0, 0), 'count': 69}
{'_id': datetime.datetime(2017, 4, 15, 0, 0), 'count': 69}
{'_id': datetime.datetime(2017, 5, 3, 0, 0), 'count': 64}
{'_id': datetime.datetime(2017, 3, 28, 0, 0), 'count': 54}
{'_id': datetime.datetime(2017, 5, 13, 0, 0), 'count': 54}
{'_id': datetime.datetime(2017, 4, 19, 0, 0), 'count': 50}
{'_id': datetime.datetime(2017, 4, 5, 0, 0), 'count': 49}
{'_id': datetime.datetime(2017, 4, 7, 0, 0), 'count': 39}
{'_id': datetime.datetime(2017, 4, 17, 0, 0), 'count': 38

### g
Find the average surface temperature (°C) for each day . You are required to only display average surface temperature (°C) and the date in the output.

In [20]:
import pymongo


def question_g():
    connection = pymongo.MongoClient("localhost", 27017)
    database = connection["fit5148_assignment_db"]
    hotspot = database["hotspot_record"]

    records = hotspot.aggregate([
        {"$group": {"_id": "$date", "average_surface_temperature": {"$avg": "$surface_temperature_celcius"}}},
        {"$sort": {"average_surface_temperature": -1}}
    ])

    for record in records:
        print(record)

    return records


question_g()

{'_id': datetime.datetime(2017, 3, 12, 0, 0), 'average_surface_temperature': 88.2}
{'_id': datetime.datetime(2017, 3, 18, 0, 0), 'average_surface_temperature': 79.33333333333333}
{'_id': datetime.datetime(2017, 10, 15, 0, 0), 'average_surface_temperature': 72.66666666666667}
{'_id': datetime.datetime(2017, 6, 20, 0, 0), 'average_surface_temperature': 71.16666666666667}
{'_id': datetime.datetime(2017, 12, 14, 0, 0), 'average_surface_temperature': 70.0}
{'_id': datetime.datetime(2017, 3, 10, 0, 0), 'average_surface_temperature': 69.375}
{'_id': datetime.datetime(2017, 5, 1, 0, 0), 'average_surface_temperature': 68.4}
{'_id': datetime.datetime(2017, 3, 25, 0, 0), 'average_surface_temperature': 66.0}
{'_id': datetime.datetime(2017, 3, 14, 0, 0), 'average_surface_temperature': 65.6}
{'_id': datetime.datetime(2017, 3, 19, 0, 0), 'average_surface_temperature': 65.57142857142857}
{'_id': datetime.datetime(2017, 3, 7, 0, 0), 'average_surface_temperature': 64.0}
{'_id': datetime.datetime(2017, 9